# Cálculo Vencimento Gestores

In [1]:
import os
import re
import json
import win32com.client
import datetime

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from utils_ccd import get_connection, get_informacoes_processo

from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate

from functools import partial

from pydantic import BaseModel, Field

from docxtpl import DocxTemplate
from num2words import num2words


In [2]:
from dotenv import load_dotenv
load_dotenv()

llm = AzureChatOpenAI(model_name="gpt-4o")

from typing import Optional

class PessoaMultaAcordao(BaseModel):
    nome: Optional[str] = Field(default=None, description="Nome da pessoa que foi multada")
    cpf: Optional[str] = Field(default=None, description="CPF da pessoa que foi multada, em formato XXXXXXXXXXX, sem pontos ou traços")
    orgao: Optional[str] = Field(default=None, description="Órgão ao qual a pessoa multada pertence")
    percentual_valor: Optional[str] = Field(default=None, description="Percentual ou valor da multa aplicada, se houver")
    ano_aplicacao_multa: Optional[int] = Field(default=None, description="Ano em que a multa na época do ato que deu causa à multa")
    cargo_gestor: Optional[str] = Field(default=None, description="Cargo do gestor multado")
    descricao: Optional[str] = Field(default=None, description="Trecho do acórdão que menciona a multa")

class PessoasMultasAcordao(BaseModel):
    pessoas: list[PessoaMultaAcordao] = Field(description="Lista de pessoas com multas percentuais identificadas no acórdão")

struct_llm = llm.with_structured_output(
    schema=PessoasMultasAcordao
)

In [ ]:
sql_gestores = """
SELECT CONCAT(p.numero_processo, '/', p.ano_processo) as processo, p.codigo_tipo_processo, p.assunto, 
r.nome, pm.Descricao as marcador, p.IdProcesso as id_processo
FROM processo.dbo.Processos p 
	INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON p.IdProcesso = pmp.IdProcesso
	INNER JOIN processo.dbo.Pro_Marcador pm ON pm.IdMarcador = pmp.IdMarcador 
	INNER JOIN processo.dbo.Relator r ON r.codigo = p.codigo_relator 
WHERE setor_atual = 'CCD' 
	AND lower(pm.Descricao) = 'CÁLCULO VENCIMENTOS DO GESTOR'
"""
processos_gestores = pd.read_sql(sql_gestores, get_connection())

In [45]:
sql_pessoas = f''' 
SELECT CONCAT(p.numero_processo, '/', p.ano_processo) as processo, gp.Documento as documento, gp.Nome as nome
FROM processo.dbo.Pro_ProcessosResponsavelDespesa prd 
	INNER JOIN processo.dbo.Processos p ON prd.IdProcesso = p.IdProcesso
	INNER JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = prd.IdPessoa 
	WHERE CONCAT(p.numero_processo, '/', p.ano_processo) IN ({', '.join([f"'{x}'" for x in processos_gestores.processo.to_list()])}); 
'''
df_pessoas = pd.read_sql(sql_pessoas, get_connection())

def interpolate_pessoas(row):
    return list(zip(row['documento'], row['nome']))

df_pessoas = df_pessoas.groupby('processo').agg({'documento': lambda x: list(x), 'nome': lambda x: list(x)}).reset_index(inplace=False)
df_pessoas['pessoas'] = df_pessoas.apply(interpolate_pessoas, axis=1)

In [46]:
def get_pessoa_multa_acordao(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa acordãos do TCE que contem multas.
        Sua tarefa é identificar o nome da pessoa que foi multada e o valor da multa, quando for o caso de multa percentual sobre vencimentos de um gestor.
        O texto do é o seguinte:
        "{input}"
       
        Sua resposta:
        """)
        chain = prompt | struct_llm
        return chain.invoke(texto)
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None


In [47]:
processos_gestores.columns

Index(['processo', 'codigo_tipo_processo', 'assunto', 'nome', 'marcador',
       'id_processo'],
      dtype='object')

In [48]:
def pessoas_string(pessoas):
    if not pessoas:
        return 'Nenhuma pessoa identificada.'
    return f''' Pessoas do processo:
        {', '.join([f"{p[1]}, CPF: {p[0]}" for p in pessoas]) if pessoas else 'Nenhuma pessoa identificada.'}

    '''
    
processos_gestores['pessoas'] = processos_gestores.processo.map(df_pessoas.set_index('processo')['pessoas'])
processos_gestores.pessoas.fillna('', inplace=True)
processos_gestores['pessoas'] = processos_gestores.pessoas.apply(pessoas_string)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_15368\3695440228.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processos_gestores.pessoas.fillna('', inplace=True)


In [49]:
processos_gestores.head()

,processo,codigo_tipo_processo,assunto,nome,marcador,id_processo,pessoas
0,000263/2019,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,494976,Pessoas do processo:\n José Júlio Fern...
1,003123/2022,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,567105,Pessoas do processo:\n Antônio Carlos ...
2,003940/2019,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE A ATRA...",FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,505409,Pessoas do processo:\n EMANUEL DA SILV...
3,004322/2019,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO EXE...",FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,505816,Pessoas do processo:\n JOAO NOLASCO NE...
4,006833/2019,APR,APURAÇÃO DE RESPONSABILIDADE - PODER EXECUTIVO...,PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,512930,Pessoas do processo:\n PEDRO AUGUSTO L...


In [18]:
infos_processo = get_informacoes_processo('004820/2020', get_connection())
infos_processo['setor'] = infos_processo['setor'].str.lower()
infos_processo['setor'] = infos_processo['setor'].str.strip()
if 'secsc' in infos_processo['setor'].to_list() or 'secpc' in infos_processo['setor'].to_list():
    texto = '\n'.join(infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].texto.to_list())
else:
    proc = infos_processo[infos_processo['setor'].str.contains('proc')]
    if len(proc) > 0:
        texto = '\n'.join(proc.texto.to_list())

c:\Users\05911205424\Documents\Dev\ccd\scripts\utils_ccd.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [ ]:
for i, row in processos_gestores.iterrows():
    print(f"{i}: {row['processo']} - {row['nome']} - {row['assunto']}")
    texto = row['pessoas']
    infos_processo = get_informacoes_processo(row['processo'], get_connection())
    if 'secsc' in infos_processo['setor'].to_list() or 'secpc' in infos_processo['setor'].to_list():
        texto = '\n'.join(infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].texto.to_list())
    else:
        proc = infos_processo[infos_processo['setor'].str.contains('proc')]
        if len(proc) > 0:
            texto = '\n'.join(proc.texto.to_list())
    if not texto:
        print("Nenhum texto encontrado para o processo.")
        processos_gestores.loc[i, 'multas_pessoas'] = None
        continue
    
    ret = get_pessoa_multa_acordao(texto)
    if ret:
        print(f"Pessoas encontradas")
        processos_gestores.loc[i, 'multas_pessoas'] = ret.model_dump_json()
    else:
        print("Nenhuma multa encontrada.")
        processos_gestores.loc[i, 'multas_pessoas'] = None

    

In [70]:
processos_gestores.to_excel('db/processos_gestores_multa.xlsx', index=False)

# Análise

In [3]:
processos_gestores = pd.read_excel('db/processos_gestores_multa.xlsx')

In [4]:
processos_gestores.head()

,processo,codigo_tipo_processo,assunto,nome,marcador,id_processo,pessoas,multas_pessoas,cpf_subsidio,pessoa_subsidio,valor_subsidio,ano_subsidio
0,000263/2019,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,494976,Pessoas do processo:\n José Júlio Fern...,"{""pessoas"":[{""nome"":""José Júlio Fernandes Neto...",5.966878e+10,José Júlio Fernandes Neto,6262.33,2014.0
1,000084/2021,EXE,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,542898,Nenhuma pessoa identificada.,"{""pessoas"":[]}",NaN,NaN,NaN,NaN
2,004555/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",FRANCISCO POTIGUAR CAVALCANTI JÚNIOR,CÁLCULO VENCIMENTOS DO GESTOR,538501,Pessoas do processo:\n FULGÊNCIO TEIXE...,"{""pessoas"":[{""nome"":""Clemente Gurgel de Amorim...",NaN,NaN,NaN,NaN
3,003953/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",ANTONIO ED SOUZA SANTANA,CÁLCULO VENCIMENTOS DO GESTOR,536056,Pessoas do processo:\n SALOMÃO GOMES D...,"{""pessoas"":[{""nome"":""Salomão Gomes de Oliveira...",NaN,NaN,NaN,NaN
4,004820/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,539327,Pessoas do processo:\n JUAREZ BEZERRA ...,"{""pessoas"":[{""nome"":""JUAREZ BEZERRA DE AZEVEDO...",NaN,NaN,NaN,NaN


# Cálculo pelo Siai Pessoal

In [6]:
processos_gestores = pd.read_excel('db/processos_gestores_multa.xlsx')

In [91]:
processos_gestores[processos_gestores['processo'] == '006552/2019'].iloc[0]['multas_pessoas']

'{"pessoas":[{"nome":"LUIZ BENES LEOCÁDIO DE ARAÚJO","cpf":"40665429487","orgao":"PREFEITURA MUNICIPAL DE LAJES/RN","percentual_valor":"5% dos vencimentos anuais","ano_aplicacao_multa":2015,"cargo_gestor":null,"descricao":"Aplicação de multa ao Sr. LUIZ BENES LEOCÁDIO DE ARAÚJO, equivalente a 5% (cinco por cento) dos seus vencimentos anuais à época, em virtude do atraso, por 2 (dois) dias"}]}'

In [7]:
sql_vencimentos = """
SELECT
    qfr.ano,
    qfr.mes,
    s.cpf COLLATE SQL_Latin1_General_CP1_CI_AS as cpf,
   	s.nome COLLATE SQL_Latin1_General_CP1_CI_AS as nome_servidor,
    icc.DescItemContraCheque as descricao_rubrica,
    cci.valor as valor_rubrica,
	o.idOrgao as id_orgao,
    rtrim(o.codigo) as codigo_orgao,
    o.nome COLLATE SQL_Latin1_General_CP1_CI_AS as nome_orgao,
    NULL as codigo_orgao_superior,
    NULL as tipo_orgao_natureza,
    s.matricula COLLATE SQL_Latin1_General_CP1_CI_AS,
    s.datanascimento,
    convert(char(10), s.[dataadmissao], 126) as data_admissao,
    convert(char(10), s.[datadesligamento], 126)  as data_desligamento,
    s.cargo COLLATE SQL_Latin1_General_CP1_CI_AS as cargo,
    s.lotacao COLLATE SQL_Latin1_General_CP1_CI_AS as lotacao,
    s.pne COLLATE SQL_Latin1_General_CP1_CI_AS,
    s.dedicacaoexclusiva COLLATE SQL_Latin1_General_CP1_CI_AS as de,
    s.cargahoraria as ch,
    upper(fi.[descformaingresso]) COLLATE SQL_Latin1_General_CP1_CI_AS as forma_ingresso,
    upper(rj.[descregimejuridico]) COLLATE SQL_Latin1_General_CP1_CI_AS as regime_juridico,
    upper(v.[descricaovinculo]) COLLATE SQL_Latin1_General_CP1_CI_AS as vinculo,
    upper(sf.[descsituacaofuncional]) COLLATE SQL_Latin1_General_CP1_CI_AS as situacao_funcional,
    --fp.idfpfolhapagamento as id_folha,
    upper(tfp.desctipofolha) as tipo_folha,
    cc.idfpcontracheque as id_contracheque,
    cast(s.totaldesconto as numeric(32,2)) as total_descontos,
    cast(s.totalvantagem as numeric(32,2)) as total_vantagens,
    cc.datainclusao AS data_inclusao,
	0 as retificacao
FROM 
	-- remessa
    [processo].[dbo].siaidp_arquivo as a
	inner join [processo].[dbo].orgaos as o on a.codigoorgao = o.codigo

	-- folha de pagamento
	inner join [processo].[dbo].siaidp_fpremessa as rfp on a.idarquivo = rfp.idarquivo
	inner join [processo].[dbo].siaidp_fpfolhapagamento as fp on rfp.idfpremessa = fp.idfpremessa
	inner join [processo].[dbo].[siaidp_fptipofolha] as tfp on fp.idfptipofolha = tfp.idfptipofolha
	inner join [processo].[dbo].siaidp_fpcontracheque as cc on fp.idfpfolhapagamento = cc.idfpfolhapagamento
	inner join [processo].[dbo].siaidp_fpcontrachequeitem as cci on cci.idfpcontracheque = cc.idfpcontracheque
	inner join [processo].[dbo].SiaiDp_FpItemContraCheque icc on cci.idfpitemcontracheque = icc.idfpitemcontracheque
	
	-- servidor
	inner join [processo].[dbo].siaidp_qfremessa as qfr on a.idarquivo = qfr.idarquivo
	inner join [processo].[dbo].siaidp_qfservidor as s on s.idqfremessa = qfr.idqfremessa and s.cpf = cc.cpf
	left join [processo].[dbo].[siaidp_qfformaingresso] as fi on s.[idqfformaingresso] = fi.[idqfformaingresso]
	left join [processo].[dbo].[siaidp_qfregimejuridico] as rj on s.[idqfregimejuridico] = rj.[idqfregimejuridico]
    left join [processo].[dbo].[siaidp_qfsituacaofuncional] as sf on s.[idqfsituacaofuncional] = sf.[idqfsituacaofuncional]
	left join [processo].[dbo].[siaidp_qfvinculo] as v on s.[idqfvinculo] = v.[idqfvinculo]

WHERE 
    a.inativo is null
    AND s.CPF = '{}' and qfr.ano = {} 
"""

In [8]:
def has_cpf(row):
    mp = row['multas_pessoas']
    if not mp:
        return False
    try:
        pessoas = PessoasMultasAcordao.model_validate_json(mp)
        all_true = []
        for p in pessoas.pessoas:
            all_true.append(p.cpf and p.percentual_valor and p.ano_aplicacao_multa)
        return any(all_true)
    except Exception as e:
        print(f"Error processing multas_pessoas: {e}")
        return False

In [9]:
valid_cpfs = processos_gestores[processos_gestores.apply(lambda row: has_cpf(row), axis=1)]

In [10]:
len(valid_cpfs), len(processos_gestores)

(14, 27)

In [11]:
def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

In [12]:
cpfs_dicts = flatten([json.loads(row['multas_pessoas'])['pessoas'] for i,row in processos_gestores.iterrows()])

In [ ]:
dfs_pessoas = {}
cpfs_processados = []
for pessoa in cpfs_dicts:
    cpf = pessoa['cpf']
    ano = pessoa['ano_aplicacao_multa']
    if not cpf or not ano:
        print(f"Skipping invalid CPF or year for {pessoa}")
        continue
    print(f"Processing CPF: {cpf}, Ano: {ano}")
    sql = sql_vencimentos.format(cpf, ano)
    dfs_pessoas[cpf] = pd.read_sql(sql, get_connection())
    cpfs_processados.append(cpf)

In [88]:
dfs_pessoas['40665429487']

,ano,mes,cpf,nome_servidor,descricao_rubrica,valor_rubrica,id_orgao,codigo_orgao,nome_orgao,codigo_orgao_superior,...,vinculo,situacao_funcional,tipo_folha,id_contracheque,total_descontos,total_vantagens,data_inclusao,retificacao,is_vencimento,is_subsidio
0,2015,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Vencimento Básico ...,17025.66,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12291480,9801.52,21120.02,2016-03-10 16:19:54.837,0,True,False
1,2015,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Auxílios e benefícios ...,1200.00,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12291480,9801.52,21120.02,2016-03-10 16:19:54.837,0,False,False
2,2015,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Anuênio ...,2894.36,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12291480,9801.52,21120.02,2016-03-10 16:19:54.837,0,False,False
3,2015,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Previdência (Regime Próprio) ...,2191.20,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12291480,9801.52,21120.02,2016-03-10 16:19:54.837,0,False,False
4,2015,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Imposto de Renda Retido na Fonte (IRRF) ...,4006.07,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12291480,9801.52,21120.02,2016-03-10 16:19:54.837,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2015,11,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Auxílios e benefícios ...,1200.00,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,11965692,9801.52,21120.02,2016-02-22 10:54:14.400,0,False,False
68,2015,11,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Anuênio ...,2894.36,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,11965692,9801.52,21120.02,2016-02-22 10:54:14.400,0,False,False
69,2015,11,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Previdência (Regime Próprio) ...,2191.20,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,11965692,9801.52,21120.02,2016-02-22 10:54:14.400,0,False,False
70,2015,11,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Imposto de Renda Retido na Fonte (IRRF) ...,4006.07,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,11965692,9801.52,21120.02,2016-02-22 10:54:14.400,0,False,False


In [54]:
def get_vencimentos(cpf):
    if cpf not in dfs_pessoas:
        print(f"CPF {cpf} not found in dataframes.")
        return pd.DataFrame()
    df = dfs_pessoas[cpf]
    if df.empty:
        print(f"No data found for CPF {cpf}.")
        return pd.DataFrame()
    df['data_inclusao'] = pd.to_datetime(df['data_inclusao'])
    df['mes'] = pd.to_datetime(df[['ano', 'mes']].assign(day=1))
    df['mes'] = df['mes'].dt.strftime('%Y-%m')
    return df


def has_person(row, cpf):
    for pessoa in json.loads(row)['pessoas']:
        if pessoa['cpf'] == cpf:
            return True
    return False


def get_texto_acordaos(pessoa):
    infos_processo = get_informacoes_processo(pessoa['processo'], get_connection())
    if 'secsc' in infos_processo['setor'].to_list() or 'secpc' in infos_processo['setor'].to_list():
        return '\n'.join(infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].texto.to_list())
    else:
        proc = infos_processo[infos_processo['setor'].str.contains('proc')]
        if len(proc) > 0:
            return '\n'.join(proc.texto.to_list())
    return None

def get_pessoa_percentual_multa(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa acordãos do TCE que contem multas.
        Sua tarefa é identificar o percentual da multa sobre vencimentos do multado.
        O texto do é o seguinte:
        "{input}"
       
        Responda apenas o percentual no formato 0.1:
        """)
        chain = prompt | llm
        return chain.invoke(texto)
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None
    
def generate_pdf_office(doc_path: str, output_dir: str) -> str:
    # Ensure paths are absolute
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.abspath(output_dir)
    pdf_path = os.path.join(output_dir, os.path.splitext(os.path.basename(doc_path))[0] + '.pdf')

    word = win32com.client.Dispatch("Word.Application")
    word.Visible = False

    try:
        doc = word.Documents.Open(doc_path)
        doc.SaveAs(pdf_path, FileFormat=17)  # 17 = wdFormatPDF
        doc.Close()
    finally:
        word.Quit()

    return pdf_path

import gender_guesser.detector as gender

def detectar_genero(nome: str) -> str:
    """
    Retorna o gênero provável de um nome próprio.
    Saídas possíveis: 'masculino', 'feminino', 'indefinido'
    """
    d = gender.Detector(case_sensitive=False)
    resultado = d.get_gender(nome.split()[0])  # pega só o primeiro nome

    if resultado in ["male", "mostly_male"]:
        return "masculino"
    elif resultado in ["female", "mostly_female"]:
        return "feminino"
    else:
        return "indefinido"

def get_texto_pessoa(nome: str) -> str:
    inicio = 'do Sr.' if detectar_genero(nome.split()[0]) == 'masculino' else 'da Sra.'
    nome = [p.capitalize() for p in nome.split()]
    return f"{inicio} {' '.join(nome).replace('De', 'de').replace('Da', 'da').replace('Dos', 'dos').replace('Das', 'das')}"

def criar_valor_extenso(x):
        if x and x > 1:
            f = lambda c: num2words(int(c), lang='pt_BR')
            return ' reais e '.join([f(y) for y in str('{:.2f}'.format(x)).split('.')]) + ' centavos'
        else:
            return ''

def converter_valor(x):
    if x:
        return locale.currency(x, grouping=True) + ' (' + criar_valor_extenso(x) + ')'
    else:
        return 'R$ 0,00 (zero)'
    
def extract_num_slash_num(text):
    match = re.search(r'.*\b(ACÓRDÃO No. \d+/\d+)\b.*', text)
    if match:
        return match.group(1)
    return None



In [ ]:
dict_calculo_vencimentos = dict()
for cpf, df in dfs_pessoas.items():
    df['is_vencimento'] = df['descricao_rubrica'].apply(lambda x: bool('vencimento' in x.strip().lower()))
    df['is_subsidio'] = df['descricao_rubrica'].apply(lambda x: bool(re.match(r'subs.dio', x.strip().lower())))
    vencimentos = df[df['is_vencimento']]
    subsidios = df[df['is_subsidio']]

    partial_has_person = partial(has_person, cpf=cpf)
    processo_pessoa = processos_gestores[processos_gestores['multas_pessoas'].apply(partial_has_person)]
    
    if processo_pessoa.empty:
        continue
   
    valor_cpf = {}

    valor_cpf['processo'] = processo_pessoa['processo'].values[0]
    valor_cpf['assunto'] = processo_pessoa['assunto'].values[0]
    valor_cpf['relator'] = processo_pessoa['nome'].values[0]

    if not df.empty:
        valor_cpf['nome'] = df.iloc[-1]['nome_servidor'].strip()
        valor_cpf['ano'] = int(df.iloc[-1]['ano'])
        valor_cpf['orgao'] = df.iloc[-1]['nome_orgao'].strip()

    if not vencimentos.empty:
        vencimento = vencimentos.iloc[-1]['valor_rubrica']
        valor_cpf['vencimento'] = float(vencimento)

    if not subsidios.empty:
        subsidio = df[df['is_subsidio']].iloc[-1]['valor_rubrica']
        valor_cpf['subsidio'] = float(subsidio)

    percentual_multa = get_pessoa_percentual_multa(get_texto_acordaos(valor_cpf))

    if percentual_multa is not None:
        valor_cpf['percentual_multa'] = percentual_multa

    if valor_cpf:
        dict_calculo_vencimentos[cpf] = valor_cpf

    

In [56]:
infos_processo = get_informacoes_processo(pessoa['processo'], get_connection())
for _, row in infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].iterrows():
    texto = infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].iloc[0]['texto']
    if 'vencimentos anuais' in texto:
        print(extract_num_slash_num(texto))

ACÓRDÃO No. 60/2021


In [ ]:
import pickle

with open('db/dict_calculo_vencimentos.pkl', 'wb') as f:
    pickle.dump(dict_calculo_vencimentos, f)

# Criação dos arquivos

In [18]:
dict_calculo_vencimentos = pd.read_pickle('db/dict_calculo_vencimentos.pkl')

In [ ]:
for cpf, pessoa in dict_calculo_vencimentos.items():
    infos_processo = get_informacoes_processo(pessoa['processo'], get_connection())
    for _, row in infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].iterrows():
        texto = infos_processo[infos_processo['setor'].isin(['secsc', 'secpc'])].iloc[0]['texto']
        if 'vencimentos anuais' in texto:
            acordao_num = extract_num_slash_num(texto)
            pessoa['acordao'] = acordao_num.capitalize()

    dict_calculo_vencimentos[cpf] = pessoa

In [84]:
def get_texto_multa(pessoa) -> str:
    vencimento = pessoa.get('vencimento', None)
    subsidio = pessoa.get('subsidio', None)
    total = vencimento + subsidio if vencimento and subsidio else None
    ano = pessoa.get('ano', None)

    total_real = converter_valor(total) if total else 'R$ 0,00'
    vencimento_real = converter_valor(vencimento) if vencimento else 'R$ 0,00'
    subsidio_real = converter_valor(subsidio) if subsidio else 'R$ 0,00'

    total_anual = converter_valor(total * 12) if total else 'R$ 0,00'
    vencimento_anual = converter_valor(vencimento * 12) if vencimento else 'R$ 0,00'
    subsidio_anual = converter_valor(subsidio * 12) if subsidio else 'R$ 0,00'

    percentual = float(pessoa['percentual_multa'].content)

    if vencimento and subsidio:
        texto_base = f'''no ano de {ano}, o gestor recebia um total de {total_real} como vencimentos mensais, sendo {vencimento_real} em vencimentos e {subsidio_real} em subsídios.\
          Assim, seus rendimentos anuais eram de {total_anual}. \
            Sobre esse valor incide a multa percentual de {int(percentual * 100)}% aplicada pelo TCE, resultando em um total de {converter_valor(percentual * (total * 12))}.'''
    elif vencimento and not subsidio:
        texto_base = f'''no ano de {ano}, o gestor recebia {vencimento_real} em vencimentos mensais. Assim, seus rendimentos anuais eram de {vencimento_anual}.\
    Sobre esse valor incide a multa percentual de {int(percentual * 100)}% aplicada pelo TCE, resultando em um total de {converter_valor(percentual * (vencimento * 12))}.'''
    elif not vencimento and subsidio:
        texto_base = f'''no ano de {ano}, o gestor recebia {subsidio_real} em subsídios mensais. Assim, seus rendimentos anuais eram de {subsidio_anual}.\
    Sobre esse valor incide a multa percentual de {int(percentual * 100)}% aplicada pelo TCE, resultando em um total de {converter_valor(percentual * (subsidio * 12))}.'''

    return texto_base

In [85]:
import traceback
for cpf, pessoa in dict_calculo_vencimentos.items():
    doc = DocxTemplate("templates/calculo_vencimentos.docx")

    try:
        texto_pessoa = get_texto_pessoa(pessoa['nome'])
        texto_multa = get_texto_multa(pessoa)
        context = {
            'processo': pessoa['processo'],
            'assunto': pessoa['assunto'],
            'pessoa': texto_pessoa,
            'texto_multa': texto_multa,
            'relator': pessoa['relator'],
            'acordao': pessoa['acordao']
        }
        processo_underline = pessoa['processo'].replace('/', '_')

        doc.render(context)
        doc.save(f"saidas/calculo_vencimentos/{processo_underline}_{pessoa['nome'].split()[0]}.docx")
        generate_pdf_office(f"saidas/calculo_vencimentos/{processo_underline}_{pessoa['nome'].split()[0]}.docx", f"saidas/calculo_vencimentos/")
        print(f"Gerado arquivo para o processo {pessoa['processo']}")
    except Exception as e:
        print(f"Erro ao gerar arquivo para o processo {pessoa['processo']}: {e}")
        traceback.print_exc()

Gerado arquivo para o processo 000263/2019
Erro ao gerar arquivo para o processo 003953/2020: 'nome'
Erro ao gerar arquivo para o processo 003953/2020: 'nome'


Traceback (most recent call last):
  File "C:\Users\05911205424\AppData\Local\Temp\ipykernel_18052\768994342.py", line 6, in <module>
    texto_pessoa = get_texto_pessoa(pessoa['nome'])
                                    ~~~~~~^^^^^^^^
KeyError: 'nome'
Traceback (most recent call last):
  File "C:\Users\05911205424\AppData\Local\Temp\ipykernel_18052\768994342.py", line 6, in <module>
    texto_pessoa = get_texto_pessoa(pessoa['nome'])
                                    ~~~~~~^^^^^^^^
KeyError: 'nome'


Gerado arquivo para o processo 006552/2019
Gerado arquivo para o processo 000548/2019
Gerado arquivo para o processo 007694/2019
Gerado arquivo para o processo 004233/2020
Gerado arquivo para o processo 004332/2019
Erro ao gerar arquivo para o processo 000278/2019: 'acordao'
Erro ao gerar arquivo para o processo 701137/2011: 'nome'


Traceback (most recent call last):
  File "C:\Users\05911205424\AppData\Local\Temp\ipykernel_18052\768994342.py", line 14, in <module>
    'acordao': pessoa['acordao']
               ~~~~~~^^^^^^^^^^^
KeyError: 'acordao'
Traceback (most recent call last):
  File "C:\Users\05911205424\AppData\Local\Temp\ipykernel_18052\768994342.py", line 6, in <module>
    texto_pessoa = get_texto_pessoa(pessoa['nome'])
                                    ~~~~~~^^^^^^^^
KeyError: 'nome'


Gerado arquivo para o processo 003123/2022
Gerado arquivo para o processo 004322/2019
Gerado arquivo para o processo 000420/2019
Gerado arquivo para o processo 004330/2019
Gerado arquivo para o processo 003940/2019


In [65]:
texto_multa

'no ano de 2015, o gestor percebia R$ 5.000,00 (cinco mil reais e zero centavos) em subsídios mensais. Assim, seus rendimentos anuais eram de R$ 60.000,00 (sessenta mil reais e zero centavos).    Sobre esse valor incide a multa percentual de 30.0% aplicada pelo TCE, resultando em um total de R$ 1.500,00 (mil e quinhentos reais e zero centavos).\n        '

In [41]:
texto_multas

'no ano de 2015, o gestor percebia R$ 17.025,66 (dezessete mil e vinte e cinco reais e sessenta e seis centavos) em vencimentos mensais. Assim, seus rendimentos anuais eram de R$ 204.307,92 (duzentos e quatro mil, trezentos e sete reais e noventa e dois centavos).    Sobre esse valor incide a multa percentual de 5.0% aplicada pelo TCE, resultando em um total de R$ 851,28 (oitocentos e cinquenta e um reais e vinte e oito centavos).\n        '

In [37]:
dict_calculo_vencimentos[cpf]

{'processo': '003953/2020',
 'assunto': 'APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO DE 2016 (INFRAÇÕES A LRF)                                                                                                                                                                                       ',
 'relator': 'ANTONIO ED SOUZA SANTANA',
 'percentual_multa': AIMessage(content='0.3', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1029, 'total_tokens': 1033, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_3eed281ddb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filt

In [32]:
dict_calculo_vencimentos

{'59668784472': {'processo': '000263/2019',
  'assunto': 'APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PROCESSO N.º 6568                                                                                                                                                                                                   ',
  'relator': 'FRANCISCO POTIGUAR CAVALCANTI JÚNIOR',
  'nome': 'JOSE JULIO FERNANDES NETO',
  'ano': 2014,
  'orgao': 'SECRETARIA DE ESTADO DA SAÚDE PUBLICA',
  'vencimento': 5862.18,
  'percentual_multa': AIMessage(content='0.3', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1001, 'total_tokens': 1005, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_3eed281ddb', 'prompt_filter_results': [{'prom

In [38]:
dfs_pessoas[cpf]

,ano,mes,cpf,nome_servidor,descricao_rubrica,valor_rubrica,id_orgao,codigo_orgao,nome_orgao,codigo_orgao_superior,...,vinculo,situacao_funcional,tipo_folha,id_contracheque,total_descontos,total_vantagens,data_inclusao,retificacao,is_vencimento,is_subsidio
